# 6

## Task 1

In [ ]:
import numpy as np

# Hidden states
states = ['Rainy', 'Sunny']
n_states = len(states)

# Observations
observations = ['no umbrella', 'umbrella', 'no umbrella', 'umbrella', 'no umbrella', 'no umbrella']
n_observations = len(observations)

# Markov chain transition probabilities
transition_prob = np.array([[0.7, 0.3],    # P(Rainy->Rainy, Rainy->Sunny)
                            [0.4, 0.6]])   # P(Sunny->Rainy, Sunny->Sunny)

# Sensor model (emission probabilities)
emission_prob = np.array([[0.9, 0.1],    # P(no umbrella|Rainy), P(umbrella|Rainy)
                          [0.2, 0.8]])   # P(no umbrella|Sunny), P(umbrella|Sunny)

# Initial state probability distribution
initial_prob = np.array([0.5, 0.5])  # P(Rainy), P(Sunny)

# Viterbi algorithm
def viterbi(obs, states, start_prob, trans_prob, emit_prob):
    n_states = len(states)
    n_obs = len(obs)
    viterbi_matrix = np.zeros((n_states, n_obs))
    path_matrix = np.zeros((n_states, n_obs), dtype=int)

    # Initialization step
    viterbi_matrix[:, 0] = start_prob * emit_prob[:, obs[0]]

    # Recursion step
    for t in range(1, n_obs):
        for s in range(n_states):
            prob = viterbi_matrix[:, t-1] * trans_prob[:, s] * emit_prob[s, obs[t]]
            viterbi_matrix[s, t] = np.max(prob)
            path_matrix[s, t] = np.argmax(prob)

    # Termination step
    best_path = np.zeros(n_obs, dtype=int)
    best_path[-1] = np.argmax(viterbi_matrix[:, -1])

    for t in range(n_obs-2, -1, -1):
        best_path[t] = path_matrix[best_path[t+1], t+1]

    best_sequence = [states[state] for state in best_path]
    return best_sequence

# Convert observations to indices
obs_indices = [0 if obs == 'no umbrella' else 1 for obs in observations]

# Run Viterbi algorithm
most_likely_sequence = viterbi(obs_indices, states, initial_prob, transition_prob, emission_prob)

print("Most likely sequence of states:")
print(most_likely_sequence)


Most likely sequence of states:
['Rainy', 'Sunny', 'Rainy', 'Sunny', 'Rainy', 'Rainy']


In [ ]:
import numpy as np

# Define the states and observations
states = ['Rainy', 'Sunny']
n_states = len(states)

observations = ['no umbrella', 'umbrella', 'no umbrella', 'umbrella', 'no umbrella', 'no umbrella']
n_observations = len(observations)

# Transition probabilities (Markov chain)
transition_prob = np.array([[0.7, 0.3],    # P(Rainy -> Rainy), P(Rainy -> Sunny)
                            [0.4, 0.6]])   # P(Sunny -> Rainy), P(Sunny -> Sunny)

# Emission probabilities (Sensor model)
emission_prob = np.array([[0.9, 0.1],    # P(no umbrella | Rainy), P(umbrella | Rainy)
                          [0.2, 0.8]])   # P(no umbrella | Sunny), P(umbrella | Sunny)

# Initial state probabilities
initial_prob = np.array([0.5, 0.5])  # P(Rainy), P(Sunny)

# Viterbi algorithm
def viterbi(obs, states, start_prob, trans_prob, emit_prob):
    n_states = len(states)
    n_obs = len(obs)

    # Initialize the Viterbi matrix and path matrix
    viterbi_matrix = np.zeros((n_states, n_obs))
    path_matrix = np.zeros((n_states, n_obs), dtype=int)

    # Initialization step
    viterbi_matrix[:, 0] = start_prob * emit_prob[:, obs[0]]

    # Recursion step
    for t in range(1, n_obs):
        for s in range(n_states):
            prob = viterbi_matrix[:, t-1] * trans_prob[:, s] * emit_prob[s, obs[t]]
            viterbi_matrix[s, t] = np.max(prob)
            path_matrix[s, t] = np.argmax(prob)

    # Termination step
    best_path = np.zeros(n_obs, dtype=int)
    best_path[-1] = np.argmax(viterbi_matrix[:, -1])

    for t in range(n_obs-2, -1, -1):
        best_path[t] = path_matrix[best_path[t+1], t+1]

    best_sequence = [states[state] for state in best_path]
    return best_sequence

# Convert observations to indices (0: no umbrella, 1: umbrella)
obs_indices = [0 if obs == 'no umbrella' else 1 for obs in observations]

# Run Viterbi algorithm
most_likely_sequence = viterbi(obs_indices, states, initial_prob, transition_prob, emission_prob)

print("Most likely sequence of hidden states:")
print(most_likely_sequence)


Most likely sequence of hidden states:
['Rainy', 'Sunny', 'Rainy', 'Sunny', 'Rainy', 'Rainy']


## Task 2

In [ ]:
import pyAgrum as gum

# Create a Bayesian Network
bn = gum.BayesNet('StudentMarks')

# Add nodes (variables)
exam_level = bn.add(gum.LabelizedVariable('exam_level', 'Exam Level', 2))  # 0: Easy, 1: Difficult
iq_level = bn.add(gum.LabelizedVariable('iq_level', 'IQ Level', 2))        # 0: Low, 1: High
aptitude_score = bn.add(gum.LabelizedVariable('aptitude_score', 'Aptitude Score', 2))  # 0: Low, 1: High
marks = bn.add(gum.LabelizedVariable('marks', 'Marks', 2))                # 0: Fail, 1: Pass
admission = bn.add(gum.LabelizedVariable('admission', 'Admission', 2))    # 0: No, 1: Yes

# Add arcs (dependencies)
bn.addArc(exam_level, marks)
bn.addArc(iq_level, marks)
bn.addArc(iq_level, aptitude_score)
bn.addArc(marks, admission)

# Define the CPTs (Conditional Probability Tables)
bn.cpt(exam_level).fillWith([0.5, 0.5])  # P(Exam Level) [Easy, Difficult]
bn.cpt(iq_level).fillWith([0.5, 0.5])    # P(IQ Level) [Low, High]

# P(Aptitude Score | IQ Level)
bn.cpt(aptitude_score)[{'iq_level': 0}] = [0.7, 0.3]  # Low IQ -> Low Aptitude Score
bn.cpt(aptitude_score)[{'iq_level': 1}] = [0.4, 0.6]  # High IQ -> High Aptitude Score

# P(Marks | Exam Level, IQ Level)
bn.cpt(marks)[{'exam_level': 0, 'iq_level': 0}] = [0.8, 0.2]  # Easy, Low IQ -> Pass
bn.cpt(marks)[{'exam_level': 0, 'iq_level': 1}] = [0.6, 0.4]  # Easy, High IQ -> Pass
bn.cpt(marks)[{'exam_level': 1, 'iq_level': 0}] = [0.3, 0.7]  # Difficult, Low IQ -> Pass
bn.cpt(marks)[{'exam_level': 1, 'iq_level': 1}] = [0.1, 0.9]  # Difficult, High IQ -> Pass

# P(Admission | Marks)
bn.cpt(admission)[{'marks': 0}] = [0.9, 0.1]  # Fail -> Admission unlikely
bn.cpt(admission)[{'marks': 1}] = [0.4, 0.6]  # Pass -> Admission more likely

# Perform inference
model = gum.LazyPropagation(bn)
model.makeInference()

# Case 1: Exam Difficult, Low IQ, Low Aptitude Score, Probability of passing and getting admission
model.setEvidence({'exam_level': 1, 'iq_level': 0, 'aptitude_score': 0})
prob_case_1 = model.posterior('admission')[1]
print(f"Case 1: Probability of passing and securing admission: {prob_case_1:.4f}")

# Case 2: Exam Easy, High IQ, High Aptitude Score, Probability of failing and not getting admission
model.setEvidence({'exam_level': 0, 'iq_level': 1, 'aptitude_score': 1})
prob_case_2 = model.posterior('admission')[0]
print(f"Case 2: Probability of failing and not securing admission: {prob_case_2:.4f}")


Case 1: Probability of passing and securing admission: 0.4500
Case 2: Probability of failing and not securing admission: 0.7000


In [ ]:
import pyAgrum as gum

# Create a Bayesian Network
bn = gum.BayesNet('StudentMarks')

# Add nodes (variables)
exam_level = bn.add(gum.LabelizedVariable('exam_level', 'Exam Level', 2))  # 0: Easy, 1: Difficult
iq_level = bn.add(gum.LabelizedVariable('iq_level', 'IQ Level', 2))        # 0: Low, 1: High
aptitude_score = bn.add(gum.LabelizedVariable('aptitude_score', 'Aptitude Score', 2))  # 0: Low, 1: High
marks = bn.add(gum.LabelizedVariable('marks', 'Marks', 2))                # 0: Fail, 1: Pass
admission = bn.add(gum.LabelizedVariable('admission', 'Admission', 2))    # 0: No, 1: Yes

# Add arcs (dependencies)
bn.addArc(exam_level, marks)
bn.addArc(iq_level, marks)
bn.addArc(iq_level, aptitude_score)
bn.addArc(marks, admission)

# Define the CPTs (Conditional Probability Tables)
bn.cpt(exam_level).fillWith([0.5, 0.5])  # P(Exam Level) [Easy, Difficult]
bn.cpt(iq_level).fillWith([0.5, 0.5])    # P(IQ Level) [Low, High]

# P(Aptitude Score | IQ Level)
bn.cpt(aptitude_score)[{'iq_level': 0}] = [0.7, 0.3]  # Low IQ -> Low Aptitude Score
bn.cpt(aptitude_score)[{'iq_level': 1}] = [0.4, 0.6]  # High IQ -> High Aptitude Score

# P(Marks | Exam Level, IQ Level)
bn.cpt(marks)[{'exam_level': 0, 'iq_level': 0}] = [0.8, 0.2]  # Easy, Low IQ -> Pass
bn.cpt(marks)[{'exam_level': 0, 'iq_level': 1}] = [0.6, 0.4]  # Easy, High IQ -> Pass
bn.cpt(marks)[{'exam_level': 1, 'iq_level': 0}] = [0.3, 0.7]  # Difficult, Low IQ -> Pass
bn.cpt(marks)[{'exam_level': 1, 'iq_level': 1}] = [0.1, 0.9]  # Difficult, High IQ -> Pass

# P(Admission | Marks)
bn.cpt(admission)[{'marks': 0}] = [0.9, 0.1]  # Fail -> Admission unlikely
bn.cpt(admission)[{'marks': 1}] = [0.4, 0.6]  # Pass -> Admission more likely

# Perform inference
model = gum.LazyPropagation(bn)
model.makeInference()

# Case 1: Exam Difficult, Low IQ, Low Aptitude Score, Probability of passing and getting admission
model.setEvidence({'exam_level': 1, 'iq_level': 0, 'aptitude_score': 0})
prob_case_1 = model.posterior('admission')[1]
print(f"Case 1: Probability of passing and securing admission: {prob_case_1:.4f}")

# Case 2: Exam Easy, High IQ, High Aptitude Score, Probability of failing and not getting admission
model.setEvidence({'exam_level': 0, 'iq_level': 1, 'aptitude_score': 1})
prob_case_2 = model.posterior('admission')[0]
print(f"Case 2: Probability of failing and not securing admission: {prob_case_2:.4f}")


Case 1: Probability of passing and securing admission: 0.4500
Case 2: Probability of failing and not securing admission: 0.7000
